## Reporte Practica 2: Procesamiento de datos <br />Carlos Espinoza - B92786

### <strong>Limpia de Datos</strong>

Es la identificación y corrección de errores dentro del dataset que pueden tener un impacto negativo en el modelo de predicción

Dentro de los muchos errores que se pueden encontrar en un conjunto de datos  están las <strong>columnas que no aportan</strong> mucha información al modelado y las <strong>filas</strong> que se encuentran <strong>repetidas</strong>


#### Identificación de columnas con un único valor
- Estas columnas no aportan datos relevantes para el modelo de predicción
<hr/>

Código para detectar cuales son las columnas que deben ser removidas:

In [ ]:

# summarize the number of unique values for each column using numpy
from urllib.request import urlopen
from numpy import loadtxt
from numpy import unique
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
# load the dataset
data = loadtxt(urlopen(path), delimiter=',')
# summarize the number of unique values in each column
for i in range(data.shape[1]):
	if len(unique(data[:,i])) == 1:
		print("Se debe remover la columna", i)
	#print(i, len(unique(data[:, i])))

Otra forma de realizar esta tarea es utilizar la función de Pandas `nunique()` para poder obtener cuántos valores diferentes hay dentro de esa columna

In [ ]:

# summarize the number of unique values for each column using numpy
from pandas import read_csv
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
# load the dataset
df = read_csv(path, header=None)
# summarize the number of unique values in each column
print(df.nunique())

#### Eliminar la columnas que contienen un único valor

Las columnas se pueden borrar fácilmente utilizando los módulos de numpy o Pandas, para ello existe la función `drop()` 

In [ ]:
# delete columns with a single unique value
from pandas import read_csv
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
# load the dataset
df = read_csv(path, header=None)
print(df.shape)
# get number of unique values for each column
counts = df.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if v == 1]
print(to_del)
# drop useless columns
df.drop(to_del, axis=1, inplace=True)
print(df.shape)

Como se puede ver en el resultado de la impresión de las dimensiones, se eliminó una columna del dataset, específicamente la 22

#### Considerar las columnas con pocos valores únicos

Como se pudo observar antes, una columna con un único valor es de poca relevancias para el modelo de predicción, por esto era factible eliminarla. En el caso de tener más de un valor diferente, pero que igual siga siendo pequeño en relación con los valores alcanzados en otras columnas, es dificil saber si aportará al proceso de modelado o no.

Dependiendo del algoritmo de predicción que se vaya a utilizar es probable que las columnas con pocos valores únicos generen problemas o resultados imprecisos.

Podemos calcular el porcentaje de valores únicos en relación a la cantidad de filas que hay en el conjunto de datos.

In [ ]:
# summarize the percentage of unique values for each column using numpy
from urllib.request import urlopen
from numpy import loadtxt
from numpy import unique
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
# load the dataset
data = loadtxt(urlopen(path), delimiter=',')
# summarize the number of unique values in each column
for i in range(data.shape[1]):
	num = len(unique(data[:, i]))
	percentage = float(num) / data.shape[0] * 100
	print('%d, %d, %.1f%%' % (i, num, percentage))

De esta forma podemos datos una idea de cuales podrían ser la columnas que nos pueden dar problemas con aquellos algoritmos que sean más sensibles a la varianza.

Además, podemos hacer cambios en el código para ver cuales son aquellas columnas que se encuentran debajo del 1%

In [ ]:
# summarize the percentage of unique values for each column using numpy
from urllib.request import urlopen
from numpy import loadtxt
from numpy import unique
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
# load the dataset
data = loadtxt(urlopen(path), delimiter=',')
# summarize the number of unique values in each column
for i in range(data.shape[1]):
	num = len(unique(data[:, i]))
	percentage = float(num) / data.shape[0] * 100
	if percentage < 1:
		print('%d, %d, %.1f%%' % (i, num, percentage))

Como resultado vemos que 11 de las 50 columnas dentro del conjunto de datos están por debajo del 1% en valores únicos. Ahora, esto únicamente nos dice que estos datos en específico requieren de un análisis más a fondo para determinar si es viable si borrado o no

#### Remover las columnas que tienen poca varianza

Otra forma de atacar el problema de la eliminación de columnas es ver cual es la varianza de la columna. Recordemos que la varianza es estática y puede ser utlizada como `filtro` para identificar columnas que deben ser eliminadas del conjunto de datos. 

Siguiendo con el ejemplo anterior, se puede ver que la columna 22 tiene una varianza de 0.0 pues esta solo tiene un dato único, mientras que las demás columnas con pocos valores únicos van a resultar con un grado de varianza cercano a 0

In [ ]:
# example of apply the variance threshold
from pandas import read_csv
from sklearn.feature_selection import VarianceThreshold
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
# load the dataset
df = read_csv(path, header=None)
# split data into inputs and outputs
data = df.values
X = data[:, :-1]
y = data[:, -1]
print(X.shape, y.shape)
# define the transform
transform = VarianceThreshold()
# transform the input data
X_sel = transform.fit_transform(X)
print(X_sel.shape)

In [ ]:
# explore the effect of the variance thresholds on the number of selected features
from numpy import arange
from pandas import read_csv
from sklearn.feature_selection import VarianceThreshold
from matplotlib import pyplot
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
# load the dataset
df = read_csv(path, header=None)
# split data into inputs and outputs
data = df.values
X = data[:, :-1]
y = data[:, -1]
print(X.shape, y.shape)
# define thresholds to check
thresholds = arange(0.0, 0.55, 0.05)
# apply transform with each threshold
results = list()
for t in thresholds:
	# define the transform
	transform = VarianceThreshold(threshold=t)
	# transform the input data
	X_sel = transform.fit_transform(X)
	# determine the number of input features
	n_features = X_sel.shape[1]
	print('>Threshold=%.2f, Features=%d' % (t, n_features))
	# store the result
	results.append(n_features)
# plot the threshold vs the number of selected features
pyplot.plot(thresholds, results)
pyplot.show()

Con este plot y con el resultado obtenido por consola se puede ver la cantidad de características o columnas que superan cierto grado de varianza

#### Identificar filas que contengan datos duplicados

Las filas que tienen datos iguales probablemente son poco útiles y hasta peligrosos pues llevan a una mala evaluación del modelo.

Para detectar posibles duplicados dentro de un conjunto de datos, Pandas, tiene una función `duplicate()` para poder verificar si este es el caso.

In [ ]:
# locate rows of duplicate data
from pandas import read_csv
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
# load the dataset
df = read_csv(path, header=None)
# calculate duplicates
dups = df.duplicated()
# report if there are any duplicates
print(dups.any())
# list all duplicate rows
print(df[dups])

Como podemos ver en el resultado, Pandas se encargó de mostrar cuales fueron los datos duplicados. Veremos como eliminar estos datos

#### Eliminar filas que contengan duplicados

Probablemente todas las filas duplicadas deban ser eliminadas previo a un modelado. Para realizar esta tarea se utilizará la función de Pandas `drop_duplicates()`.

In [ ]:
# delete rows of duplicate data from the dataset
from pandas import read_csv
# define the location of the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
# load the dataset
df = read_csv(path, header=None)
print(df.shape)
# delete duplicate rows
df.drop_duplicates(inplace=True)
print(df.shape)

Con este resultado podemos ver el estado del conjunto de datos previo al borrado (150 filas) y después de este (147 filas), que serían las encontradas en la sección anterior

### <strong>Comentarios sobre el ejercicio</strong>

Antes de esta práctica había trabajado muy poco con la parte de limpieza de datos, únicamente con las tareas más sencillas como lo son el borrado de duplicados. Me parece interesante esta nueva perspectiva que adquirí, de pensar que además de borrar filas, que suele ser lo más común, también se analice la utilidad de ciertas columnas o `features` dentro del dataset. También, me pareció interesante ver algunas de las técnicas utilizadas para determinar si una columna es de relevancia para el análisis, porque vimos que no por tener pocos valores únicos necesariamente se tiene que eliminar ese `feature`, en casos así se debe aplicar un análisis de varianzas para ayudar a tomar la decisión. 

Como mencioné anteriormente pocas veces he tenido que trabajar con estas técnicas pero siento que, aunque vimos pocas en el tutorial, son claves para llegar a tener un conjunto de datos mucho más limpio y además, son de gran ayuda para los analistas, pues nos dan una mejor visión del conjunto con el que estamos trabajando. 